In [83]:
### My implementaion of simple matrix multiplication (O(n^3))
function simple_matrix_multiplication(A::Matrix{T}, B::Matrix{T}) where T<:AbstractFloat
    n, m = size(A)
    m2, p = size(B)
    
    C = zeros(T, n, p)
    
    @inbounds @simd for i in 1:n
        @inbounds @simd for j in 1:p
            @inbounds @simd for k in 1:m
                C[i, j] += A[i, k] * B[k, j]
            end
        end
    end
    
    return C
end


simple_matrix_multiplication (generic function with 1 method)

In [97]:
### My implementaion of simple matrix multiplication (O(n^3))
function simple_matrix_multiplication(A::Matrix{T}, B::Matrix{T}) where T<:AbstractFloat
    n, m = size(A)
    m2, p = size(B)
    
    C = zeros(T, n, p)
    
    @inbounds @simd for i in 1:n
        @inbounds @simd for j in 1:p
            #@inbounds @simd for k in 1:m
                C[i, j] = sum(@view A[i, :] .* @view B[:, j])
                #C[i, j]=mapreduce(+, *,  A[i, :],  B[:, j])
            #end
        end
    end
    
    return C
end


simple_matrix_multiplication (generic function with 1 method)

In [96]:
mapreduce(+, *,  [1,2,3,4], [10,20,30,40])

351384

In [5]:
#padding the matrix to make it n*n where n=2^k, this is the only suitable size for Strassen algorithm
function pad_matrix(M1::Matrix{T}, M2::Matrix{T} ) where T<:AbstractFloat
    n=2^ceil(Int, log2(max(size(M1)[1], size(M1)[2], size(M2)[2])))
    padded_M1 = zeros(T, n, n)
    original_size = size(M1)
    padded_M1[1:original_size[1], 1:original_size[2]] = M1

    padded_M2 = zeros(T, n, n)
    original_size = size(M2)
    padded_M2[1:original_size[1], 1:original_size[2]] = M2

    return padded_M1, padded_M2
end

pad_matrix (generic function with 1 method)

In [52]:
#example A and B for multipllication
n = 128
m=128
k=128
A = rand(Float32, n, m);
B = rand(Float32, m, k);

In [98]:
@btime C = simple_matrix_multiplication(A, B);

  109.082 ms (114690 allocations: 28.81 MiB)


In [11]:
padded_A, padded_B = pad_matrix(A, B)
@time C_padded = strassen(padded_A, padded_B)
C = C_padded[1:size(A)[1], 1:size(B)[2]];

  0.075309 seconds (627.46 k allocations: 57.024 MiB, 39.51% gc time)


In [14]:
padded_A, padded_B = pad_matrix(A, B)
@time C_padded = hybrid_strassen(padded_A, padded_B)
C = C_padded[1:size(A)[1], 1:size(B)[2]];

  0.000236 seconds (34 allocations: 580.047 KiB)


In [17]:
@time C_padded = hybrid_strassen_gpu(padded_A, padded_B)
C = C_padded[1:size(A)[1], 1:size(B)[2]];

  0.003339 seconds (899 allocations: 313.234 KiB)


In [57]:
@time C=A*B;

  0.000285 seconds (2 allocations: 64.047 KiB)


In [24]:
@code_llvm A*B

;  @ C:\workdir\usr\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:111 within `*`
; Function Attrs: uwtable
define nonnull {}* @"julia_*_4139"({}* noundef nonnull align 16 dereferenceable(40) %0, {}* noundef nonnull align 16 dereferenceable(40) %1) #0 {
L72:
  %gcframe45 = alloca [3 x {}*], align 16
  %gcframe45.sub = getelementptr inbounds [3 x {}*], [3 x {}*]* %gcframe45, i64 0, i64 0
  %2 = bitcast [3 x {}*]* %gcframe45 to i8*
  call void @llvm.memset.p0i8.i64(i8* align 16 %2, i8 0, i64 24, i1 true)
  %3 = call {}*** inttoptr (i64 140715123397680 to {}*** ()*)() #7
;  @ C:\workdir\usr\share\julia\stdlib\v1.10\LinearAlgebra\src\matmul.jl:113 within `*`
; ┌ @ array.jl:190 within `size`
   %4 = bitcast [3 x {}*]* %gcframe45 to i64*
   store i64 4, i64* %4, align 16
   %5 = getelementptr inbounds [3 x {}*], [3 x {}*]* %gcframe45, i64 0, i64 1
   %6 = bitcast {}** %5 to {}***
   %7 = load {}**, {}*** %3, align 8
   store {}** %7, {}*** %6, align 8
   %8 = bitcast {}*** %3 to {}***


In [25]:
@simd

LoadError: LoadError: MethodError: no method matching var"@simd"(::LineNumberNode, ::Module)

Closest candidates are:
  var"@simd"(::LineNumberNode, ::Module, !Matched::Any, !Matched::Any)
   @ Base simdloop.jl:131
  var"@simd"(::LineNumberNode, ::Module, !Matched::Any)
   @ Base simdloop.jl:127

in expression starting at c:\Users\babazadehmorteza\Desktop\MthClub\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X16sZmlsZQ==.jl:1

In [29]:
begin 
    import Pkg
    Pkg.add("SIMD")
    using SIMD
end

   Resolving package versions...
    Updating `C:\Users\babazadehmorteza\Desktop\MthClub\Project.toml`
  [fdea26ae] + SIMD v3.5.0
    Updating `C:\Users\babazadehmorteza\Desktop\MthClub\Manifest.toml`
  [fdea26ae] + SIMD v3.5.0


In [63]:
begin
    b = ones(10_000)
    a = 2 .* ones(10_000)
    @btime for i in 1:10_000
        a[i] *= b[i]
    end
end

  1.151 ms (58467 allocations: 913.55 KiB)


In [62]:
begin
    b = ones(10_000)
    a = 2 .* ones(10_000)
    @btime for i in 1:10_000
        a[i] += b[i]
    end
end

  1.133 ms (58467 allocations: 913.55 KiB)
